In [1]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 

In [18]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [19]:
train_df.head()
test_id = test_df['ID_code']

In [4]:
print(train_df['target'].value_counts())
unsatisfied_cnt = train_df[train_df['target'] == 1].target.count()
total_cnt = train_df.target.count()
print('unsatisfied 비율은 {0:.2f}'.format((unsatisfied_cnt / total_cnt)))

0    179902
1     20098
Name: target, dtype: int64
unsatisfied 비율은 0.10


In [5]:
train_df.describe( )

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [6]:
#object=id, int=target
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [7]:
# var3 피처 값 대체 및 ID 피처 드롭
train_df.drop('ID_code',axis=1 , inplace=True)
test_df.drop('ID_code',axis=1 , inplace=True)

In [8]:
# 피처 세트와 레이블 세트분리. 레이블 컬럼은 DataFrame의 맨 마지막에 위치해 컬럼 위치 -1로 분리
X_features = train_df.iloc[:, 1:]
y_labels = train_df.iloc[:, -0]
print('피처 데이터 shape:{0}'.format(X_features.shape))

피처 데이터 shape:(200000, 200)


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=0)
train_cnt = y_train.count()
test_cnt = y_test.count()
print('학습 세트 Shape:{0}, 테스트 세트 Shape:{1}'.format(X_train.shape , X_test.shape))

print(' 학습 세트 레이블 값 분포 비율')
print(y_train.value_counts()/train_cnt)
print('\n 테스트 세트 레이블 값 분포 비율')
print(y_test.value_counts()/test_cnt)

학습 세트 Shape:(160000, 200), 테스트 세트 Shape:(40000, 200)
 학습 세트 레이블 값 분포 비율
0    0.899475
1    0.100525
Name: target, dtype: float64

 테스트 세트 레이블 값 분포 비율
0    0.89965
1    0.10035
Name: target, dtype: float64


In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=0)
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (160000, 200) (160000,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (287832, 200) (287832,)
SMOTE 적용 후 레이블 값 분포: 
 1    143916
0    143916
dtype: int64


# xgboost

In [12]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# n_estimators는 500으로, random state는 예제 수행 시마다 동일 예측 결과를 위해 설정. 
xgb_clf = XGBClassifier(max_depth=2,
                              n_estimators=10000,
                              colsample_bytree=0.3,
                              learning_rate=0.02,
                              objective='binary:logistic', 
                              n_jobs=-1)

# 성능 평가 지표를 auc로, 조기 중단 파라미터는 100으로 설정하고 학습 수행. 
xgb_clf.fit(X_train, y_train, early_stopping_rounds=100,
            eval_metric="auc", eval_set=[(X_train, y_train), (X_test, y_test)])

xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(xgb_roc_score))

[0]	validation_0-auc:0.576457	validation_1-auc:0.573421
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 100 rounds.
[1]	validation_0-auc:0.611066	validation_1-auc:0.605624
[2]	validation_0-auc:0.617639	validation_1-auc:0.612419
[3]	validation_0-auc:0.63542	validation_1-auc:0.62951
[4]	validation_0-auc:0.631889	validation_1-auc:0.627037
[5]	validation_0-auc:0.640703	validation_1-auc:0.64021
[6]	validation_0-auc:0.643179	validation_1-auc:0.642729
[7]	validation_0-auc:0.648618	validation_1-auc:0.649294
[8]	validation_0-auc:0.650657	validation_1-auc:0.651946
[9]	validation_0-auc:0.649731	validation_1-auc:0.651088
[10]	validation_0-auc:0.65168	validation_1-auc:0.653297
[11]	validation_0-auc:0.650404	validation_1-auc:0.652376
[12]	validation_0-auc:0.656293	validation_1-auc:0.656161
[13]	validation_0-auc:0.656765	validation_1-auc:0.657278
[14]	validation_0-auc:0.658935	validation_1-auc:0.658698
[

[141]	validation_0-auc:0.739861	validation_1-auc:0.730226
[142]	validation_0-auc:0.740897	validation_1-auc:0.731164
[143]	validation_0-auc:0.741388	validation_1-auc:0.731629
[144]	validation_0-auc:0.742728	validation_1-auc:0.732921
[145]	validation_0-auc:0.743562	validation_1-auc:0.733824
[146]	validation_0-auc:0.744614	validation_1-auc:0.734816
[147]	validation_0-auc:0.745127	validation_1-auc:0.735397
[148]	validation_0-auc:0.746172	validation_1-auc:0.73622
[149]	validation_0-auc:0.746972	validation_1-auc:0.737143
[150]	validation_0-auc:0.746951	validation_1-auc:0.736785
[151]	validation_0-auc:0.747295	validation_1-auc:0.737048
[152]	validation_0-auc:0.748034	validation_1-auc:0.737675
[153]	validation_0-auc:0.748227	validation_1-auc:0.737949
[154]	validation_0-auc:0.74898	validation_1-auc:0.738756
[155]	validation_0-auc:0.74892	validation_1-auc:0.738671
[156]	validation_0-auc:0.749373	validation_1-auc:0.739228
[157]	validation_0-auc:0.750319	validation_1-auc:0.740254
[158]	validation_

[283]	validation_0-auc:0.798601	validation_1-auc:0.785759
[284]	validation_0-auc:0.799036	validation_1-auc:0.786205
[285]	validation_0-auc:0.799088	validation_1-auc:0.786304
[286]	validation_0-auc:0.799326	validation_1-auc:0.786566
[287]	validation_0-auc:0.798973	validation_1-auc:0.786164
[288]	validation_0-auc:0.799253	validation_1-auc:0.786248
[289]	validation_0-auc:0.799275	validation_1-auc:0.786385
[290]	validation_0-auc:0.799536	validation_1-auc:0.786646
[291]	validation_0-auc:0.799921	validation_1-auc:0.787045
[292]	validation_0-auc:0.799849	validation_1-auc:0.786898
[293]	validation_0-auc:0.800151	validation_1-auc:0.787207
[294]	validation_0-auc:0.800551	validation_1-auc:0.787503
[295]	validation_0-auc:0.800754	validation_1-auc:0.787848
[296]	validation_0-auc:0.801014	validation_1-auc:0.788049
[297]	validation_0-auc:0.801399	validation_1-auc:0.788366
[298]	validation_0-auc:0.801799	validation_1-auc:0.788653
[299]	validation_0-auc:0.802096	validation_1-auc:0.788865
[300]	validati

[425]	validation_0-auc:0.824156	validation_1-auc:0.808571
[426]	validation_0-auc:0.824234	validation_1-auc:0.808602
[427]	validation_0-auc:0.824383	validation_1-auc:0.808661
[428]	validation_0-auc:0.824554	validation_1-auc:0.808838
[429]	validation_0-auc:0.824759	validation_1-auc:0.809132
[430]	validation_0-auc:0.824926	validation_1-auc:0.809293
[431]	validation_0-auc:0.825177	validation_1-auc:0.809485
[432]	validation_0-auc:0.82532	validation_1-auc:0.809729
[433]	validation_0-auc:0.82541	validation_1-auc:0.809893
[434]	validation_0-auc:0.82563	validation_1-auc:0.810071
[435]	validation_0-auc:0.825788	validation_1-auc:0.810283
[436]	validation_0-auc:0.825938	validation_1-auc:0.810339
[437]	validation_0-auc:0.82586	validation_1-auc:0.810256
[438]	validation_0-auc:0.82604	validation_1-auc:0.810422
[439]	validation_0-auc:0.826304	validation_1-auc:0.810589
[440]	validation_0-auc:0.826462	validation_1-auc:0.810746
[441]	validation_0-auc:0.826502	validation_1-auc:0.810802
[442]	validation_0-

[567]	validation_0-auc:0.842031	validation_1-auc:0.824763
[568]	validation_0-auc:0.842061	validation_1-auc:0.824733
[569]	validation_0-auc:0.84216	validation_1-auc:0.824833
[570]	validation_0-auc:0.842212	validation_1-auc:0.82489
[571]	validation_0-auc:0.842297	validation_1-auc:0.824954
[572]	validation_0-auc:0.842354	validation_1-auc:0.825033
[573]	validation_0-auc:0.84236	validation_1-auc:0.825044
[574]	validation_0-auc:0.842321	validation_1-auc:0.824989
[575]	validation_0-auc:0.84243	validation_1-auc:0.825069
[576]	validation_0-auc:0.842502	validation_1-auc:0.825125
[577]	validation_0-auc:0.842635	validation_1-auc:0.825214
[578]	validation_0-auc:0.842702	validation_1-auc:0.825194
[579]	validation_0-auc:0.842804	validation_1-auc:0.825333
[580]	validation_0-auc:0.842918	validation_1-auc:0.825412
[581]	validation_0-auc:0.843036	validation_1-auc:0.825517
[582]	validation_0-auc:0.843114	validation_1-auc:0.825565
[583]	validation_0-auc:0.84323	validation_1-auc:0.825683
[584]	validation_0-

...

[8273]	validation_0-auc:0.933148	validation_1-auc:0.894029
[8274]	validation_0-auc:0.933149	validation_1-auc:0.894028
[8275]	validation_0-auc:0.933152	validation_1-auc:0.894031
[8276]	validation_0-auc:0.933154	validation_1-auc:0.894036
[8277]	validation_0-auc:0.933157	validation_1-auc:0.894035
[8278]	validation_0-auc:0.933159	validation_1-auc:0.894035
[8279]	validation_0-auc:0.93316	validation_1-auc:0.894037
[8280]	validation_0-auc:0.933163	validation_1-auc:0.894035
[8281]	validation_0-auc:0.933165	validation_1-auc:0.894033
[8282]	validation_0-auc:0.933166	validation_1-auc:0.894033
[8283]	validation_0-auc:0.933168	validation_1-auc:0.894032
[8284]	validation_0-auc:0.93317	validation_1-auc:0.894033
[8285]	validation_0-auc:0.933171	validation_1-auc:0.894032
[8286]	validation_0-auc:0.933173	validation_1-auc:0.894033
[8287]	validation_0-auc:0.933175	validation_1-auc:0.894028
[8288]	validation_0-auc:0.933177	validation_1-auc:0.894029
[8289]	validation_0-auc:0.93318	validation_1-auc:0.894024


[8413]	validation_0-auc:0.933461	validation_1-auc:0.894098
[8414]	validation_0-auc:0.933465	validation_1-auc:0.894097
[8415]	validation_0-auc:0.933465	validation_1-auc:0.894096
[8416]	validation_0-auc:0.933468	validation_1-auc:0.894095
[8417]	validation_0-auc:0.933472	validation_1-auc:0.894095
[8418]	validation_0-auc:0.933474	validation_1-auc:0.894097
[8419]	validation_0-auc:0.933477	validation_1-auc:0.894096
[8420]	validation_0-auc:0.933479	validation_1-auc:0.894098
[8421]	validation_0-auc:0.933479	validation_1-auc:0.8941
[8422]	validation_0-auc:0.933481	validation_1-auc:0.894096
[8423]	validation_0-auc:0.933481	validation_1-auc:0.894091
[8424]	validation_0-auc:0.933483	validation_1-auc:0.894096
[8425]	validation_0-auc:0.933486	validation_1-auc:0.894096
[8426]	validation_0-auc:0.93349	validation_1-auc:0.894097
[8427]	validation_0-auc:0.933492	validation_1-auc:0.894096
[8428]	validation_0-auc:0.933495	validation_1-auc:0.894097
[8429]	validation_0-auc:0.933496	validation_1-auc:0.894096


[8553]	validation_0-auc:0.933766	validation_1-auc:0.894146
[8554]	validation_0-auc:0.933768	validation_1-auc:0.894149
[8555]	validation_0-auc:0.93377	validation_1-auc:0.894151
[8556]	validation_0-auc:0.933772	validation_1-auc:0.894151
[8557]	validation_0-auc:0.933776	validation_1-auc:0.894149
[8558]	validation_0-auc:0.933777	validation_1-auc:0.894147
[8559]	validation_0-auc:0.93378	validation_1-auc:0.894149
[8560]	validation_0-auc:0.933782	validation_1-auc:0.894148
[8561]	validation_0-auc:0.933783	validation_1-auc:0.894148
[8562]	validation_0-auc:0.933784	validation_1-auc:0.894146
[8563]	validation_0-auc:0.933787	validation_1-auc:0.894145
[8564]	validation_0-auc:0.933789	validation_1-auc:0.894141
[8565]	validation_0-auc:0.933791	validation_1-auc:0.894142
[8566]	validation_0-auc:0.933792	validation_1-auc:0.89414
[8567]	validation_0-auc:0.933793	validation_1-auc:0.894144
[8568]	validation_0-auc:0.933793	validation_1-auc:0.894145
[8569]	validation_0-auc:0.933794	validation_1-auc:0.894147


[8693]	validation_0-auc:0.934051	validation_1-auc:0.894157
[8694]	validation_0-auc:0.934053	validation_1-auc:0.894154
[8695]	validation_0-auc:0.934057	validation_1-auc:0.894158
[8696]	validation_0-auc:0.934057	validation_1-auc:0.894157
[8697]	validation_0-auc:0.934057	validation_1-auc:0.894157
[8698]	validation_0-auc:0.93406	validation_1-auc:0.894158
[8699]	validation_0-auc:0.934061	validation_1-auc:0.894153
[8700]	validation_0-auc:0.934064	validation_1-auc:0.894147
[8701]	validation_0-auc:0.934065	validation_1-auc:0.894149
[8702]	validation_0-auc:0.934066	validation_1-auc:0.89415
[8703]	validation_0-auc:0.934068	validation_1-auc:0.89415
[8704]	validation_0-auc:0.934073	validation_1-auc:0.894154
[8705]	validation_0-auc:0.934074	validation_1-auc:0.894156
[8706]	validation_0-auc:0.934075	validation_1-auc:0.894157
[8707]	validation_0-auc:0.934079	validation_1-auc:0.894161
[8708]	validation_0-auc:0.93408	validation_1-auc:0.894161
[8709]	validation_0-auc:0.934081	validation_1-auc:0.894166
[

[8833]	validation_0-auc:0.934352	validation_1-auc:0.894169
[8834]	validation_0-auc:0.934353	validation_1-auc:0.894171
[8835]	validation_0-auc:0.934355	validation_1-auc:0.894171
[8836]	validation_0-auc:0.934357	validation_1-auc:0.894171
[8837]	validation_0-auc:0.934358	validation_1-auc:0.894178
[8838]	validation_0-auc:0.934359	validation_1-auc:0.894174
[8839]	validation_0-auc:0.934362	validation_1-auc:0.894175
[8840]	validation_0-auc:0.934361	validation_1-auc:0.894175
[8841]	validation_0-auc:0.934362	validation_1-auc:0.894171
[8842]	validation_0-auc:0.934363	validation_1-auc:0.894174
[8843]	validation_0-auc:0.934364	validation_1-auc:0.894175
[8844]	validation_0-auc:0.934367	validation_1-auc:0.894176
[8845]	validation_0-auc:0.934367	validation_1-auc:0.894179
[8846]	validation_0-auc:0.93437	validation_1-auc:0.894179
[8847]	validation_0-auc:0.934372	validation_1-auc:0.894176
[8848]	validation_0-auc:0.934373	validation_1-auc:0.894176
Stopping. Best iteration:
[8748]	validation_0-auc:0.93417

#  lgboost

In [23]:
lgbm_clf = LGBMClassifier(    
    n_estimators=10000,
    bagging_fraction= 0.3,
    boost_from_average= False,
    boost= 'gbdt',
    feature_fraction= 0.041,
    learning_rate= 0.03,
    max_depth= -1,
    metric='logloss',
    min_data_in_leaf= 80,
    min_sum_hessian_in_leaf= 10.0,
    num_leaves= 13,
    num_threads= 8,
    tree_learner= 'serial',
    objective= 'binary', 
    verbosity= -1
)

evals = [(X_test, y_test)]
lgbm_clf.fit(X_train, y_train, early_stopping_rounds=100, eval_metric="auc", eval_set=evals,
                verbose=True)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1],average='macro')
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	valid_0's auc: 0.577134
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.619391
[3]	valid_0's auc: 0.65068
[4]	valid_0's auc: 0.670313
[5]	valid_0's auc: 0.674364
[6]	valid_0's auc: 0.690533
[7]	valid_0's auc: 0.709807
[8]	valid_0's auc: 0.712528
[9]	valid_0's auc: 0.71798
[10]	valid_0's auc: 0.719577
[11]	valid_0's auc: 0.727271
[12]	valid_0's auc: 0.728994
[13]	valid_0's auc: 0.738302
[14]	valid_0's auc: 0.744055
[15]	valid_0's auc: 0.752205
[16]	valid_0's auc: 0.762391
[17]	valid_0's auc: 0.768569
[18]	valid_0's auc: 0.773034
[19]	valid_0's auc: 0.774185
[20]	valid_0's auc: 0.770691
[21]	valid_0's auc: 0.769324
[22]	valid_0's auc: 0.780172
[23]	valid_0's auc: 0.781409
[24]	valid_0's auc: 0.778917
[25]	valid_0's auc: 0.783836
[26]	valid_0's auc: 0.787566
[27]	valid_0's auc: 0.786374
[28]	valid_0's auc: 0.784972
[29]	valid_0's auc: 0.787993
[30]	valid_0's auc: 0.79325
[31]	valid_0's auc: 0.794382
[32]	valid_0's auc: 0.797654
[33]	valid_0's auc: 0

[286]	valid_0's auc: 0.872383
[287]	valid_0's auc: 0.872548
[288]	valid_0's auc: 0.872615
[289]	valid_0's auc: 0.872656
[290]	valid_0's auc: 0.87296
[291]	valid_0's auc: 0.872976
[292]	valid_0's auc: 0.872895
[293]	valid_0's auc: 0.87286
[294]	valid_0's auc: 0.872861
[295]	valid_0's auc: 0.872855
[296]	valid_0's auc: 0.872867
[297]	valid_0's auc: 0.872982
[298]	valid_0's auc: 0.873045
[299]	valid_0's auc: 0.873129
[300]	valid_0's auc: 0.873225
[301]	valid_0's auc: 0.873225
[302]	valid_0's auc: 0.873182
[303]	valid_0's auc: 0.873233
[304]	valid_0's auc: 0.873334
[305]	valid_0's auc: 0.873492
[306]	valid_0's auc: 0.873687
[307]	valid_0's auc: 0.873732
[308]	valid_0's auc: 0.87367
[309]	valid_0's auc: 0.873816
[310]	valid_0's auc: 0.873937
[311]	valid_0's auc: 0.87396
[312]	valid_0's auc: 0.873923
[313]	valid_0's auc: 0.873958
[314]	valid_0's auc: 0.874087
[315]	valid_0's auc: 0.874146
[316]	valid_0's auc: 0.874154
[317]	valid_0's auc: 0.874273
[318]	valid_0's auc: 0.874322
[319]	valid_0'

[568]	valid_0's auc: 0.883106
[569]	valid_0's auc: 0.883096
[570]	valid_0's auc: 0.883082
[571]	valid_0's auc: 0.883048
[572]	valid_0's auc: 0.883095
[573]	valid_0's auc: 0.883075
[574]	valid_0's auc: 0.883123
[575]	valid_0's auc: 0.883178
[576]	valid_0's auc: 0.883193
[577]	valid_0's auc: 0.883201
[578]	valid_0's auc: 0.883215
[579]	valid_0's auc: 0.88322
[580]	valid_0's auc: 0.883277
[581]	valid_0's auc: 0.883287
[582]	valid_0's auc: 0.883274
[583]	valid_0's auc: 0.883258
[584]	valid_0's auc: 0.88333
[585]	valid_0's auc: 0.883243
[586]	valid_0's auc: 0.883208
[587]	valid_0's auc: 0.883288
[588]	valid_0's auc: 0.883289
[589]	valid_0's auc: 0.883301
[590]	valid_0's auc: 0.883368
[591]	valid_0's auc: 0.883373
[592]	valid_0's auc: 0.883441
[593]	valid_0's auc: 0.883518
[594]	valid_0's auc: 0.883555
[595]	valid_0's auc: 0.883636
[596]	valid_0's auc: 0.883555
[597]	valid_0's auc: 0.88359
[598]	valid_0's auc: 0.883636
[599]	valid_0's auc: 0.883671
[600]	valid_0's auc: 0.883704
[601]	valid_0

[853]	valid_0's auc: 0.887986
[854]	valid_0's auc: 0.888017
[855]	valid_0's auc: 0.888047
[856]	valid_0's auc: 0.888032
[857]	valid_0's auc: 0.88803
[858]	valid_0's auc: 0.888048
[859]	valid_0's auc: 0.888076
[860]	valid_0's auc: 0.888083
[861]	valid_0's auc: 0.888126
[862]	valid_0's auc: 0.888151
[863]	valid_0's auc: 0.88818
[864]	valid_0's auc: 0.88815
[865]	valid_0's auc: 0.888167
[866]	valid_0's auc: 0.888211
[867]	valid_0's auc: 0.888226
[868]	valid_0's auc: 0.888202
[869]	valid_0's auc: 0.888215
[870]	valid_0's auc: 0.888228
[871]	valid_0's auc: 0.888225
[872]	valid_0's auc: 0.888202
[873]	valid_0's auc: 0.888232
[874]	valid_0's auc: 0.88826
[875]	valid_0's auc: 0.888256
[876]	valid_0's auc: 0.888289
[877]	valid_0's auc: 0.888287
[878]	valid_0's auc: 0.888265
[879]	valid_0's auc: 0.888273
[880]	valid_0's auc: 0.88829
[881]	valid_0's auc: 0.888324
[882]	valid_0's auc: 0.888338
[883]	valid_0's auc: 0.888316
[884]	valid_0's auc: 0.888319
[885]	valid_0's auc: 0.888331
[886]	valid_0's

[1129]	valid_0's auc: 0.891112
[1130]	valid_0's auc: 0.891114
[1131]	valid_0's auc: 0.891119
[1132]	valid_0's auc: 0.891114
[1133]	valid_0's auc: 0.891149
[1134]	valid_0's auc: 0.89117
[1135]	valid_0's auc: 0.891168
[1136]	valid_0's auc: 0.891161
[1137]	valid_0's auc: 0.891158
[1138]	valid_0's auc: 0.891172
[1139]	valid_0's auc: 0.891194
[1140]	valid_0's auc: 0.891179
[1141]	valid_0's auc: 0.891174
[1142]	valid_0's auc: 0.891173
[1143]	valid_0's auc: 0.891179
[1144]	valid_0's auc: 0.891187
[1145]	valid_0's auc: 0.891208
[1146]	valid_0's auc: 0.891196
[1147]	valid_0's auc: 0.891203
[1148]	valid_0's auc: 0.891221
[1149]	valid_0's auc: 0.891236
[1150]	valid_0's auc: 0.891257
[1151]	valid_0's auc: 0.891278
[1152]	valid_0's auc: 0.89127
[1153]	valid_0's auc: 0.891267
[1154]	valid_0's auc: 0.89127
[1155]	valid_0's auc: 0.89125
[1156]	valid_0's auc: 0.891257
[1157]	valid_0's auc: 0.891275
[1158]	valid_0's auc: 0.891279
[1159]	valid_0's auc: 0.89129
[1160]	valid_0's auc: 0.891323
[1161]	valid_

[1401]	valid_0's auc: 0.892771
[1402]	valid_0's auc: 0.892787
[1403]	valid_0's auc: 0.892813
[1404]	valid_0's auc: 0.89283
[1405]	valid_0's auc: 0.892836
[1406]	valid_0's auc: 0.892829
[1407]	valid_0's auc: 0.892829
[1408]	valid_0's auc: 0.89284
[1409]	valid_0's auc: 0.892854
[1410]	valid_0's auc: 0.892847
[1411]	valid_0's auc: 0.89285
[1412]	valid_0's auc: 0.892854
[1413]	valid_0's auc: 0.892868
[1414]	valid_0's auc: 0.892863
[1415]	valid_0's auc: 0.892868
[1416]	valid_0's auc: 0.892869
[1417]	valid_0's auc: 0.892879
[1418]	valid_0's auc: 0.892878
[1419]	valid_0's auc: 0.89289
[1420]	valid_0's auc: 0.89291
[1421]	valid_0's auc: 0.89292
[1422]	valid_0's auc: 0.892918
[1423]	valid_0's auc: 0.892917
[1424]	valid_0's auc: 0.892916
[1425]	valid_0's auc: 0.892918
[1426]	valid_0's auc: 0.892928
[1427]	valid_0's auc: 0.892942
[1428]	valid_0's auc: 0.892956
[1429]	valid_0's auc: 0.89297
[1430]	valid_0's auc: 0.892965
[1431]	valid_0's auc: 0.892988
[1432]	valid_0's auc: 0.892996
[1433]	valid_0'

[1667]	valid_0's auc: 0.894143
[1668]	valid_0's auc: 0.894155
[1669]	valid_0's auc: 0.894164
[1670]	valid_0's auc: 0.894157
[1671]	valid_0's auc: 0.894165
[1672]	valid_0's auc: 0.894178
[1673]	valid_0's auc: 0.894182
[1674]	valid_0's auc: 0.894168
[1675]	valid_0's auc: 0.894168
[1676]	valid_0's auc: 0.894184
[1677]	valid_0's auc: 0.894195
[1678]	valid_0's auc: 0.894207
[1679]	valid_0's auc: 0.894195
[1680]	valid_0's auc: 0.89419
[1681]	valid_0's auc: 0.894195
[1682]	valid_0's auc: 0.894205
[1683]	valid_0's auc: 0.89422
[1684]	valid_0's auc: 0.894239
[1685]	valid_0's auc: 0.894249
[1686]	valid_0's auc: 0.894251
[1687]	valid_0's auc: 0.894255
[1688]	valid_0's auc: 0.89427
[1689]	valid_0's auc: 0.894275
[1690]	valid_0's auc: 0.894288
[1691]	valid_0's auc: 0.894292
[1692]	valid_0's auc: 0.894305
[1693]	valid_0's auc: 0.894304
[1694]	valid_0's auc: 0.894295
[1695]	valid_0's auc: 0.894304
[1696]	valid_0's auc: 0.894302
[1697]	valid_0's auc: 0.894315
[1698]	valid_0's auc: 0.894327
[1699]	vali

[1951]	valid_0's auc: 0.895254
[1952]	valid_0's auc: 0.89525
[1953]	valid_0's auc: 0.895264
[1954]	valid_0's auc: 0.895267
[1955]	valid_0's auc: 0.895268
[1956]	valid_0's auc: 0.895277
[1957]	valid_0's auc: 0.895286
[1958]	valid_0's auc: 0.895276
[1959]	valid_0's auc: 0.895279
[1960]	valid_0's auc: 0.895278
[1961]	valid_0's auc: 0.895283
[1962]	valid_0's auc: 0.895282
[1963]	valid_0's auc: 0.895262
[1964]	valid_0's auc: 0.895261
[1965]	valid_0's auc: 0.895247
[1966]	valid_0's auc: 0.895252
[1967]	valid_0's auc: 0.895252
[1968]	valid_0's auc: 0.895248
[1969]	valid_0's auc: 0.895256
[1970]	valid_0's auc: 0.895258
[1971]	valid_0's auc: 0.895264
[1972]	valid_0's auc: 0.895267
[1973]	valid_0's auc: 0.895262
[1974]	valid_0's auc: 0.895278
[1975]	valid_0's auc: 0.895289
[1976]	valid_0's auc: 0.895279
[1977]	valid_0's auc: 0.895278
[1978]	valid_0's auc: 0.895285
[1979]	valid_0's auc: 0.895275
[1980]	valid_0's auc: 0.895288
[1981]	valid_0's auc: 0.895283
[1982]	valid_0's auc: 0.895283
[1983]	va

[2241]	valid_0's auc: 0.895694
[2242]	valid_0's auc: 0.895701
[2243]	valid_0's auc: 0.895701
[2244]	valid_0's auc: 0.895694
[2245]	valid_0's auc: 0.895699
[2246]	valid_0's auc: 0.895698
[2247]	valid_0's auc: 0.8957
[2248]	valid_0's auc: 0.895702
[2249]	valid_0's auc: 0.895691
[2250]	valid_0's auc: 0.89568
[2251]	valid_0's auc: 0.895688
[2252]	valid_0's auc: 0.895694
[2253]	valid_0's auc: 0.895701
[2254]	valid_0's auc: 0.895689
[2255]	valid_0's auc: 0.895705
[2256]	valid_0's auc: 0.895707
[2257]	valid_0's auc: 0.895724
[2258]	valid_0's auc: 0.895734
[2259]	valid_0's auc: 0.895721
[2260]	valid_0's auc: 0.895716
[2261]	valid_0's auc: 0.895712
[2262]	valid_0's auc: 0.89571
[2263]	valid_0's auc: 0.895717
[2264]	valid_0's auc: 0.895721
[2265]	valid_0's auc: 0.895701
[2266]	valid_0's auc: 0.895699
[2267]	valid_0's auc: 0.895702
[2268]	valid_0's auc: 0.895704
[2269]	valid_0's auc: 0.895701
[2270]	valid_0's auc: 0.895708
[2271]	valid_0's auc: 0.895707
[2272]	valid_0's auc: 0.895706
[2273]	valid

[2528]	valid_0's auc: 0.895989
[2529]	valid_0's auc: 0.895987
[2530]	valid_0's auc: 0.895991
[2531]	valid_0's auc: 0.895987
[2532]	valid_0's auc: 0.895994
[2533]	valid_0's auc: 0.895995
[2534]	valid_0's auc: 0.895991
[2535]	valid_0's auc: 0.895991
[2536]	valid_0's auc: 0.896002
[2537]	valid_0's auc: 0.896011
[2538]	valid_0's auc: 0.896011
[2539]	valid_0's auc: 0.896015
[2540]	valid_0's auc: 0.896011
[2541]	valid_0's auc: 0.896021
[2542]	valid_0's auc: 0.89601
[2543]	valid_0's auc: 0.896001
[2544]	valid_0's auc: 0.896001
[2545]	valid_0's auc: 0.896002
[2546]	valid_0's auc: 0.895993
[2547]	valid_0's auc: 0.89601
[2548]	valid_0's auc: 0.896009
[2549]	valid_0's auc: 0.896001
[2550]	valid_0's auc: 0.896006
[2551]	valid_0's auc: 0.896006
[2552]	valid_0's auc: 0.896005
[2553]	valid_0's auc: 0.896006
[2554]	valid_0's auc: 0.896008
[2555]	valid_0's auc: 0.896013
[2556]	valid_0's auc: 0.896012
[2557]	valid_0's auc: 0.896006
[2558]	valid_0's auc: 0.896009
[2559]	valid_0's auc: 0.896003
[2560]	val

# SCORE

In [51]:
print('LGB SCORE :',roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1]))
print('XGB SCORE :',roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1]))

LGB SCORE : 0.896068956506947
XGB SCORE : 0.8941912713328614


In [54]:
print('LGB+XGB SCORE :',roc_auc_score(y_test,lgbm_clf.predict_proba(X_test)[:,1]+xgb_clf.predict_proba(X_test)[:,1]))

LGB+XGB SCORE : 0.8957526138645902


In [47]:
lgbm=lgbm_clf.predict_proba(test_df.iloc[:, 1:])[:,1]
xgbm=xgb_clf.predict_proba(test_df.iloc[:, 1:])[:,1]

In [55]:
sub_df1 = pd.DataFrame({"ID_code":test_id})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("xgboost_lgboost.csv", index=False)